# Optimize parameters for the reference case for VARIATIONAL method
- VARIATIONAL : 

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

import hvplot.xarray
import hvplot.pandas

import matplotlib.pyplot as plt
from cycler import cycler

from sstats import signals as sg
from sstats import sigp as sigp
from sstats import tseries as ts
from sstats import get_cmap_colors

import hvplot.xarray
import hvplot.pandas
import holoviews as hv

import pynsitu as pyn
import lib as lib
from lib import KEYS, raw_dir
import os
from glob import glob

import synthetic_traj as st

from synthetic_traj import (
    synthetic_traj,
    noise_irregular_sampling,
    ref_case,
    typical_case,
)

___________
# Reference case

In [ ]:
N = 20
acc_cut = 1
position_noise = 20
ntype = "white_noise"
offset_type = "svp_scripps_10"
dt_smooth = "30min"
true_key = "True_" + dt_smooth
spectral_diff = False
ref_case["spectral_diff"] = spectral_diff
print(ref_case)

In [ ]:
DF = dict()

# TRUE
t = (
    50,
    "1min",
)  # use it instead of (50, 1/24/60 because otherwise not regularly sampled
dst = synthetic_traj(t, N, **ref_case)  # u,v, ax, ay computed
dft = st.dataset2dataframe(dst).rename(columns={"draw": "id"})
DF["True_1min"] = dft

# OBSERVED

dso = noise_irregular_sampling(
    dst, t, position_noise, ntype=ntype, offset_type=offset_type, istart=4067
)
dfo = st.dataset2dataframe(dso).rename(columns={"draw": "id"})
dfo = dfo.groupby("id").apply(pyn.geo.compute_dt, time="index")

DF["Observed"] = dfo

# Interpolated True
dsti = dst.interp(time=pd.date_range(dfo.index.min(), dfo.index.max(), freq=dt_smooth))
dsti["dt"] = dsti.time.diff("time") / pd.Timedelta("1s")
dfti = st.dataset2dataframe(dsti).rename(columns={"draw": "id"})
dsti = dsti.rename({"draw": "id"})

DF["True_" + dt_smooth] = dfti

___________
# VARIATIONAL
Parameters : 
- acc_cut
- position_error = 20m
- acceleration_amplitude = 1e-5 > 5e-5
- acceleration_T in seconds = 5.5 ~T du synthetic traj ?
- time_chunk



In [ ]:
np.sqrt((dst.ax**2).mean("draw").mean())

In [ ]:
np.sqrt((dst.ay**2).mean("draw").mean())

In [ ]:
acc_cut = 1
position_error = position_noise
acceleration_amplitude = 1e-5
acceleration_T = 5.5 * 86400
time_chunk = 2

In [ ]:
DF.keys()

___________
# Acc_cut

In [ ]:
acc_cut_l = np.sort(
    [2e-4, 3e-4, 4e-4, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 2e-4, 3e-4, 5e-4, 1e-3]
)  # enp.linspace(1e-6, 1e-3, 10)
# DF0 = DF.copy()
# SMOOTHED
for accc in acc_cut_l:
    dfv = pyn.drifters.smooth_all(
        dfo,
        "variational",
        dt_smooth,
        parameters=dict(
            acc_cut=accc,
            position_error=position_error,
            acceleration_amplitude=acceleration_amplitude,
            acceleration_T=acceleration_T,
            time_chunk=time_chunk,
            acc_cut_key=("ax", "ay", "Axy"),
        ),
        spectral_diff=spectral_diff,
        geo=False,
    )
    DF0[f"acc_cut ={accc}"] = dfv
DF0.keys()

In [ ]:
d = 0
st.hvplot_DF(DF0, d, ["x", "y", "u", "v", "ax", "ay"])

In [ ]:
dfms, dse = st.diagnostics(DF0, true_key="True_30min", num=True, param_name="acc_cut")

___________
# Position error

In [ ]:
position_error_l = np.arange(5, 50, 5)
DF1 = DF.copy()
# SMOOTHED
for p in position_error_l:
    dfv = pyn.drifters.smooth_all(
        dfo,
        "variational",
        dt_smooth,
        parameters=dict(
            acc_cut=acc_cut,
            position_error=p,
            acceleration_amplitude=acceleration_amplitude,
            acceleration_T=acceleration_T,
            acc_cut_key=("ax", "ay", "Axy"),
        ),
        spectral_diff=spectral_diff,
        geo=False,
    )
    DF1[f"position_error ={p}"] = dfv
DF1.keys()

In [ ]:
DF1.keys()

In [ ]:
d = 0
st.hvplot_DF(DF1, d, ["x", "y", "u", "v", "ax", "ay"])

In [ ]:
DF1.pop("True_1min")
dfms = st.diagnostics(DF1, true_key="True_30min", num=True, param_name="position_error")

In [ ]:
DSE = st.spectrum_DF(DF1, nperseg="20D", detrend="linear")
dse = st.DSE2dse(DSE)
dsen, dsep = st.negpos_spectra(dse)


fig4, axs = plt.subplots(1, 3, figsize=(10, 5))

for l in dsen.Trajectory:
    dsen.x_y.mean("id").plot(hue="Trajectory", ax=axs[0], add_legend=True, lw=1)
    dsen.u_v.mean("id").plot(hue="Trajectory", ax=axs[1], add_legend=False, lw=1)
    dsen.ax_ay.mean("id").plot(hue="Trajectory", ax=axs[2], add_legend=False, lw=1)
for ax in axs:
    ax.grid()
    ax.set_xscale("log")
    ax.set_yscale("log")
fig4.suptitle("PSD")
fig4.tight_layout()

fig4, axs = plt.subplots(1, 3, figsize=(10, 5))

for l in dsep.Trajectory:
    dsep.x_y.mean("id").plot(hue="Trajectory", ax=axs[0], add_legend=True, lw=1)
    dsep.u_v.mean("id").plot(hue="Trajectory", ax=axs[1], add_legend=False, lw=1)
    dsep.ax_ay.mean("id").plot(hue="Trajectory", ax=axs[2], add_legend=False, lw=1)
for ax in axs:
    ax.grid()
    ax.set_xscale("log")
    ax.set_yscale("log")
fig4.suptitle("PSD")
fig4.tight_layout()

___________
# Acceleration amplitude

In [ ]:
acceleration_amplitude_l = [
    0.4e-5,
    0.5e-5,
    0.75e-5,
    1e-5,
    1.25e-5,
    1.5e-5,
    1.9e-5,
]  # np.arange(1e-5,4e-5, 2e-6)
DF2 = DF.copy()
# SMOOTHED
for aca in acceleration_amplitude_l:
    dfv = pyn.drifters.smooth_all(
        dfo,
        "variational",
        dt_smooth,
        parameters=dict(
            acc_cut=acc_cut,
            position_error=position_error,
            acceleration_amplitude=aca,
            acceleration_T=acceleration_T,
            acc_cut_key=("ax", "ay", "Axy"),
        ),
        spectral_diff=spectral_diff,
        geo=False,
    )
    DF2[f"acceleration_amplitude ={aca}"] = dfv
DF2.keys()

In [ ]:
d = 0
st.hvplot_DF(DF2, d, ["x", "y", "u", "v", "ax", "ay"])

In [ ]:
DF2.pop("True_1min")
dfms = st.diagnostics(
    DF2, true_key="True_30min", num=True, param_name="acceleration_amplitude"
)

In [ ]:
DSE = st.spectrum_DF(DF2, nperseg="20D", detrend="linear")
dse = st.DSE2dse(DSE)
dsen, dsep = st.negpos_spectra(dse)


fig4, axs = plt.subplots(1, 3, figsize=(10, 5))

for l in dsen.Trajectory:
    dsen.x_y.mean("id").plot(hue="Trajectory", ax=axs[0], add_legend=True, lw=1)
    dsen.u_v.mean("id").plot(hue="Trajectory", ax=axs[1], add_legend=False, lw=1)
    dsen.ax_ay.mean("id").plot(hue="Trajectory", ax=axs[2], add_legend=False, lw=1)
for ax in axs:
    ax.grid()
    ax.set_xscale("log")
    ax.set_yscale("log")
fig4.suptitle("PSD")
fig4.tight_layout()

fig4, axs = plt.subplots(1, 3, figsize=(10, 5))

for l in dsep.Trajectory:
    dsep.x_y.mean("id").plot(hue="Trajectory", ax=axs[0], add_legend=True, lw=1)
    dsep.u_v.mean("id").plot(hue="Trajectory", ax=axs[1], add_legend=False, lw=1)
    dsep.ax_ay.mean("id").plot(hue="Trajectory", ax=axs[2], add_legend=False, lw=1)
for ax in axs:
    ax.grid()
    ax.set_xscale("log")
    ax.set_yscale("log")
fig4.suptitle("PSD")
fig4.tight_layout()

___________
# Acceleration T

In [ ]:
acceleration_T_l = np.array([4, 4.5, 5, 5.5, 6, 7, 8]) * 86400
DF3 = DF.copy()
# SMOOTHED
for aca in acceleration_T_l:
    dfv = pyn.drifters.smooth_all(
        dfo,
        "variational",
        dt_smooth,
        parameters=dict(
            acc_cut=acc_cut,
            position_error=position_error,
            acceleration_amplitude=acceleration_amplitude,
            acceleration_T=aca,
            acc_cut_key=("ax", "ay", "Axy"),
        ),
        spectral_diff=spectral_diff,
        geo=False,
    )
    DF3[f"acceleration_T ={aca}"] = dfv
DF3.keys()

In [ ]:
d = 0
st.hvplot_DF(DF3, d, ["x", "y", "u", "v", "ax", "ay"])

In [ ]:
DF3.pop("True_1min")
dfms = st.diagnostics(DF3, true_key=true_key, num=True, param_name="acceleration_T")

In [ ]:
DSE = st.spectrum_DF(DF3, nperseg="20D", detrend="linear")
dse = st.DSE2dse(DSE)
dsen, dsep = st.negpos_spectra(dse)


fig4, axs = plt.subplots(1, 3, figsize=(10, 5))

for l in dsen.Trajectory:
    dsen.x_y.mean("id").plot(hue="Trajectory", ax=axs[0], add_legend=True, lw=0.5)
    dsen.u_v.mean("id").plot(hue="Trajectory", ax=axs[1], add_legend=False, lw=0.5)
    dsen.ax_ay.mean("id").plot(hue="Trajectory", ax=axs[2], add_legend=False, lw=0.5)
for ax in axs:
    ax.grid()
    ax.set_xscale("log")
    ax.set_yscale("log")
fig4.suptitle("PSD")
fig4.tight_layout()

fig4, axs = plt.subplots(1, 3, figsize=(10, 5))

for l in dsep.Trajectory:
    dsep.x_y.mean("id").plot(hue="Trajectory", ax=axs[0], add_legend=True, lw=0.5)
    dsep.u_v.mean("id").plot(hue="Trajectory", ax=axs[1], add_legend=False, lw=0.5)
    dsep.ax_ay.mean("id").plot(hue="Trajectory", ax=axs[2], add_legend=False, lw=0.5)
for ax in axs:
    ax.grid()
    ax.set_xscale("log")
    ax.set_yscale("log")
fig4.suptitle("PSD")
fig4.tight_layout()

_________
# 2D acceleration_T -aceeleration_amplitude
## Compute $ \frac{\langle(\alpha-\alpha_t)^2\rangle}{\langle \alpha_t^2 \rangle} $ 
 Pour acceleration amplitude = [1e-06,3e-06,5e-06,1.5e-05,2.5e-05,3.5e-05,4.5e-05,5.5e-05,6.5e-05,7.5e-05, 8.5e-05,9.5e-05]  
 
 Pour acceleration_T = np.arange(0.5,10)
 
Pour x,y rapport des variances [%] : entre (1.056e-4,1.852e-4) et (1.895e-4, 3.341e-4)-> rapport presque x2 mais reste autour de 10-4 % 
    minimun (2.5, 5e-06 , 0.5, 3e-06)  
Pour u,v rapport des variances [%] : entre (1.754e-1, 1.743e-1) et (8.43e-1, 9.652e-1) -> rapport presque x5 mais reste autour de 10-1%
    minimum (1.5 5e-06, 0.5 3e-06)  
Pour ax,ay rapport des variances [%] : entre (72.91,73.72) et (246.5, 260.5)-> 70% minimum à 260%
    minimum (1.5 5e-06, 2.5 5e-06)

In [ ]:
acc_A = [0.1e-5, 0.2e-5, 0.3e-5, 0.4e-5] + list(np.arange(0.5, 10) * 1e-5)
acc_T = [0.05, 0.1, 0.25] + list(np.arange(0.5, 10))

DSV = []
for acca in acc_A:
    DSV_ = []
    for acct in acc_T:
        dfv = pyn.drifters.smooth_all(
            dfo,
            "variational",
            dt_smooth,
            parameters=dict(
                acc_cut=acc_cut,
                position_error=position_error,
                acceleration_amplitude=acca,
                acceleration_T=acct * 86400,
                acc_cut_key=("ax", "ay", "Axy"),
            ),
            spectral_diff=spectral_diff,
            geo=False,
        )
        dfv = dfv.reset_index().set_index(["time", "id"])
        dsv = dfv.to_xarray()
        DSV_.append(dsv)

    DSV.append(xr.concat(DSV_, dim=acc_T).rename({"concat_dim": "acceleration_T"}))
ds = xr.concat(DSV, dim=acc_A).rename({"concat_dim": "acceleration_amplitude"})

In [ ]:
dsvar = (((ds - dsti).var("time")) / (dsti.var("time"))).mean("id")

In [ ]:
dsvar.max() * 100

In [ ]:
dsvar.min() * 100

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(10, 7))
axs = axs.flatten()
np.log(dsvar.x).plot(ax=axs[0])
np.log(dsvar.y).plot(ax=axs[1])
np.log(dsvar.X).plot(ax=axs[2])
np.log(dsvar.u).plot(ax=axs[3])
np.log(dsvar.U).plot(ax=axs[4])
np.log(dsvar.v).plot(ax=axs[5])
np.log(dsvar.ax).plot(ax=axs[6])
np.log(dsvar.ay).plot(ax=axs[7])
np.log(dsvar.Axy).plot(ax=axs[8])
fig.tight_layout()

In [ ]:
for v in ["x", "y", "X", "u", "v", "U", "ax", "ay", "Axy"]:
    idx = dsvar[v].argmin(dim=["acceleration_T", "acceleration_amplitude"])
    print(v)
    print(
        float(dsvar.acceleration_T[idx["acceleration_T"]].values),
        float(dsvar.acceleration_amplitude[idx["acceleration_amplitude"]].values),
        float(dsvar[v].min().values),
    )

_________
# 2D acceleration_A - position_error


In [ ]:
acc_A = [0.1e-5, 0.2e-5, 0.3e-5, 0.4e-5] + list(np.arange(0.5, 10) * 1e-5)
pos_e = [5, 10, 20, 50, 60, 70, 80, 90, 100]
acct = 1.5

DSV = []
for acca in acc_A:
    DSV_ = []
    for p in pos_e:
        dfv = pyn.drifters.smooth_all(
            dfo,
            "variational",
            dt_smooth,
            parameters=dict(
                acc_cut=acc_cut,
                position_error=p,
                acceleration_amplitude=acca,
                acceleration_T=acct * 86400,
                acc_cut_key=("ax", "ay", "Axy"),
            ),
            spectral_diff=spectral_diff,
            geo=False,
        )
        dfv = dfv.reset_index().set_index(["time", "id"])
        dsv = dfv.to_xarray()
        DSV_.append(dsv)

    DSV.append(xr.concat(DSV_, dim=pos_e).rename({"concat_dim": "position_error"}))
ds0 = xr.concat(DSV, dim=acc_A).rename({"concat_dim": "acceleration_amplitude"})

In [ ]:
dsvar0 = (((ds - dsti).var("time")) / (dsti.var("time"))).mean("id")

In [ ]:
dsvar0.max() * 100

In [ ]:
dsvar.min() * 100

In [ ]:
dsvar00 = dsvar0.where(dsvar0.position_error < 200, drop=True)

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(10, 7))
axs = axs.flatten()
np.log(dsvar00.x).plot(ax=axs[0])
np.log(dsvar00.y).plot(ax=axs[1])
np.log(dsvar00.X).plot(ax=axs[2])
np.log(dsvar00.u).plot(ax=axs[3])
np.log(dsvar00.U).plot(ax=axs[4])
np.log(dsvar00.v).plot(ax=axs[5])
np.log(dsvar00.ax).plot(ax=axs[6])
np.log(dsvar00.ay).plot(ax=axs[7])
np.log(dsvar00.Axy).plot(ax=axs[8])
fig.tight_layout()

In [ ]:
for v in ["x", "y", "X", "u", "v", "U", "ax", "ay", "Axy"]:
    idx = dsvar0[v].argmin(dim=["position_error", "acceleration_amplitude"])
    print(v)
    print(
        float(dsvar0.position_error[idx["position_error"]].values),
        float(dsvar0.acceleration_amplitude[idx["acceleration_amplitude"]].values),
        float(dsvar0[v].min().values),
    )

_________
# 2D acceleration_T - position_error


In [ ]:
acca = 0.2e-5
pos_e = [5, 10, 20, 40, 60, 80, 100]
acc_T = [0.05, 0.1, 0.25] + list(np.arange(0.5, 10))

DSV = []
for acct in acc_T:
    DSV_ = []
    for p in pos_e:
        dfv = pyn.drifters.smooth_all(
            dfo,
            "variational",
            dt_smooth,
            parameters=dict(
                acc_cut=acc_cut,
                position_error=p,
                acceleration_amplitude=acca,
                acceleration_T=acct * 86400,
                acc_cut_key=("ax", "ay", "Axy"),
            ),
            spectral_diff=spectral_diff,
            geo=False,
        )
        dfv = dfv.reset_index().set_index(["time", "id"])
        dsv = dfv.to_xarray()
        DSV_.append(dsv)

    DSV.append(xr.concat(DSV_, dim=pos_e).rename({"concat_dim": "position_error"}))
ds1 = xr.concat(DSV, dim=acc_T).rename({"concat_dim": "acceleration_T"})

In [ ]:
dsvar1 = (((ds0 - dsti).var("time")) / (dsti.var("time"))).mean("id")

In [ ]:
dsvar1.max() * 100

In [ ]:
dsvar1.min() * 100

In [ ]:
fig, axs = plt.subplots(3, 3, figsize=(10, 7))
axs = axs.flatten()
i = 0
for v in ["x", "y", "X", "u", "v", "U", "ax", "ay", "Axy"]:
    np.log(dsvar1[v]).plot(ax=axs[i])
    i += 1
fig.tight_layout()

In [ ]:
for v in ["x", "y", "X", "u", "v", "U", "ax", "ay", "Axy"]:
    idx = dsvar1[v].argmin(dim=["position_error", "acceleration_T"])
    print(v)
    print(
        float(dsvar1.position_error[idx["position_error"]].values),
        float(dsvar1.acceleration_T[idx["acceleration_T"]].values),
        float(dsvar1[v].min().values),
    )

_________
# 3D
## Compute $ \frac{\langle(\alpha-\alpha_t)^2\rangle}{\langle \alpha_t^2 \rangle} $ 


In [ ]:
acc_A = [0.1e-5, 0.2e-5, 0.3e-5, 0.4e-5] + list(np.arange(0.5, 10) * 1e-5)
pos_e = [5, 10, 20, 40, 60, 80, 100]
acc_T = [0.05, 0.1, 0.25] + list(np.arange(0.5, 10))


def f_3D():
    DSV = []
    for acca in acc_A:
        DSV_ = []
        for p in pos_e:
            DSV__ = []
            for acct in acc_T:
                dfv = pyn.drifters.smooth_all(
                    dfo,
                    "variational",
                    dt_smooth,
                    parameters=dict(
                        acc_cut=acc_cut,
                        position_error=p,
                        acceleration_amplitude=acca,
                        acceleration_T=acct * 86400,
                        acc_cut_key=("ax", "ay", "Axy"),
                    ),
                    spectral_diff=spectral_diff,
                    geo=False,
                )
                dfv = dfv.reset_index().set_index(["time", "id"])
                dsv = dfv.to_xarray()
                return dsv
                DSV__.append(dsv)
            DSV_.append(
                xr.concat(DSV__, dim=acc_T).rename({"concat_dim": "acceleration_T"})
            )
        DSV.append(xr.concat(DSV_, dim=pos_e).rename({"concat_dim": "position_error"}))
    return xr.concat(DSV, dim=acc_A).rename({"concat_dim": "acceleration_amplitude"})


# ds.to_netcdf('/Users/mdemol/code/PhD/insitu_drifters_trajectories/diagnostic_synth_traj/3D_variational.nc')

In [ ]:
ds = f_3D()

___________
# Time_chunk

In [ ]:
time_chunk_l = np.arange(2, 10)
DF4 = DF.copy()
# SMOOTHED
for t in time_chunk_l:
    dfv = pyn.drifters.smooth_all(
        dfo,
        "variational",
        dt_smooth,
        parameters=dict(
            acc_cut=acc_cut,
            position_error=position_error,
            acceleration_amplitude=acceleration_amplitude,
            acceleration_T=acceleration_T,
            time_chunk=t,
            acc_cut_key=("ax", "ay", "Axy"),
        ),
        spectral_diff=spectral_diff,
        geo=False,
    )
    DF4[f"time_chunk ={t}"] = dfv
DF4.keys()

In [ ]:
d = 0
st.hvplot_DF(DF4, d, ["x", "y", "u", "v", "ax", "ay"])

In [ ]:
# DF4.pop('True_1min')
dfms = st.diagnostics(DF4, num=True, true_key="True_30min", param_name="acceleration_T")